In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("SUPPLY CHAIN RISK PREDICTION USING GRAPH NEURAL NETWORKS")
print("="*70)

SUPPLY CHAIN RISK PREDICTION USING GRAPH NEURAL NETWORKS


In [2]:
# =========================================================================
# STEP 1: LOAD AND EXPLORE DATA
# ============================================================================
print("\n[STEP 1] Loading Dataset...")
df = pd.read_csv('sampled_10000.csv')

print(f"\nDataset Shape: {df.shape}")
print(f"\nFirst 5 rows:")
print(df.head())
print(f"\nColumn Names: {df.columns.tolist()}")
print(f"\nData Types:\n{df.dtypes}")
print(f"\n  Missing Values BEFORE Cleaning:")
missing_summary = df.isnull().sum()
print(missing_summary[missing_summary > 0])
print(f"Total missing values: {df.isnull().sum().sum()}")


[STEP 1] Loading Dataset...

Dataset Shape: (10000, 53)

First 5 rows:
       Type  Days for shipping (real)  Days for shipment (scheduled)  \
0  TRANSFER                         5                              4   
1   PAYMENT                         2                              1   
2  TRANSFER                         2                              4   
3  TRANSFER                         5                              4   
4     DEBIT                         2                              4   

   Benefit per order  Sales per customer   Delivery Status  \
0          11.090000          175.990005     Late delivery   
1           9.800000          245.000000     Late delivery   
2         117.550003          244.899994  Advance shipping   
3         118.430000          251.979996     Late delivery   
4         -21.590000          107.970001  Advance shipping   

   Late_delivery_risk  Category Id         Category Name Customer City  ...  \
0                   1           48         

In [4]:
# ==========================================================================
# STEP 2: DATA CLEANING AND PREPROCESSING
# ============================================================================
print("\n" + "="*70)
print("[STEP 2] Data Cleaning & Preprocessing...")
print("="*70)

# Store original shape
original_shape = df.shape

# Check critical columns
required_columns = ['Order City', 'Customer City', 'Late_delivery_risk']
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    raise ValueError(f"Missing required columns: {missing_cols}")

print("\n Handling Missing Values in Critical Columns...")

# Check missing values in critical columns
for col in required_columns:
    missing_count = df[col].isnull().sum()
    missing_pct = (missing_count / len(df)) * 100
    print(f"  {col}: {missing_count} missing ({missing_pct:.2f}%)")

# Strategy 1: Drop rows with missing critical columns
print("\n Dropping rows with missing critical values...")
df_clean = df.dropna(subset=required_columns)
print(f"  Rows dropped: {len(df) - len(df_clean)}")
print(f"  Remaining rows: {len(df_clean)}")

# Strategy 2: Clean city names (remove whitespace, handle case)
print("\n Cleaning city names...")
df_clean['Order City'] = df_clean['Order City'].str.strip().str.title()
df_clean['Customer City'] = df_clean['Customer City'].str.strip().str.title()

# Remove any rows where cities are empty strings after cleaning
df_clean = df_clean[
    (df_clean['Order City'].str.len() > 0) & 
    (df_clean['Customer City'].str.len() > 0)
]
print(f"  After cleaning empty strings: {len(df_clean)} rows")

# Strategy 3: Handle Late_delivery_risk - ensure it's binary
print("\n  Validating Late_delivery_risk column...")
print(f"  Unique values: {df_clean['Late_delivery_risk'].unique()}")
df_clean['Late_delivery_risk'] = df_clean['Late_delivery_risk'].astype(int)

# Ensure values are 0 or 1
invalid_risk = df_clean[~df_clean['Late_delivery_risk'].isin([0, 1])]
if len(invalid_risk) > 0:
    print(f"    Warning: {len(invalid_risk)} rows with invalid risk values, removing...")
    df_clean = df_clean[df_clean['Late_delivery_risk'].isin([0, 1])]

print(f"  Valid risk values: {df_clean['Late_delivery_risk'].value_counts().to_dict()}")

# Strategy 4: Handle Shipping Mode (if exists)
if 'Shipping Mode' in df_clean.columns:
    print("\n Handling Shipping Mode...")
    shipping_missing = df_clean['Shipping Mode'].isnull().sum()
    print(f"  Missing Shipping Mode: {shipping_missing}")
    if shipping_missing > 0:
        # Fill with 'Unknown' or most common value
        mode_value = df_clean['Shipping Mode'].mode()[0] if len(df_clean['Shipping Mode'].mode()) > 0 else 'Standard'
        df_clean['Shipping Mode'].fillna(mode_value, inplace=True)
        print(f"  Filled with: {mode_value}")

# Strategy 5: Remove duplicate edges (same Order City -> Customer City)
print("\n Handling duplicate routes...")
duplicates_before = len(df_clean)
# Keep the one with highest risk (conservative approach)
df_clean = df_clean.sort_values('Late_delivery_risk', ascending=False).drop_duplicates(
    subset=['Order City', 'Customer City'], keep='first'
)
duplicates_removed = duplicates_before - len(df_clean)
print(f"  Duplicate routes removed: {duplicates_removed}")
print(f"  Remaining unique routes: {len(df_clean)}")

# Final summary
print("\n Data Cleaning Summary:")
print(f"  Original rows: {original_shape[0]}")
print(f"  Cleaned rows: {len(df_clean)}")
print(f"  Rows removed: {original_shape[0] - len(df_clean)} ({((original_shape[0] - len(df_clean))/original_shape[0]*100):.2f}%)")
print(f"  Data retained: {(len(df_clean)/original_shape[0]*100):.2f}%")

# Check if we have enough data left
if len(df_clean) < 100:
    raise ValueError(f" Too few rows remaining ({len(df_clean)}). Check data quality!")

print(f"\n Missing Values AFTER Cleaning:")
missing_after = df_clean.isnull().sum()
if missing_after.sum() > 0:
    print(missing_after[missing_after > 0])
else:
    print("  No missing values!")

print(f"\nLate Delivery Risk Distribution:")
print(df_clean['Late_delivery_risk'].value_counts())
print(f"Late Risk Ratio: {df_clean['Late_delivery_risk'].mean():.2%}")

# Update df to use cleaned version
df = df_clean.copy()



[STEP 2] Data Cleaning & Preprocessing...

 Handling Missing Values in Critical Columns...
  Order City: 0 missing (0.00%)
  Customer City: 0 missing (0.00%)
  Late_delivery_risk: 0 missing (0.00%)

 Dropping rows with missing critical values...
  Rows dropped: 0
  Remaining rows: 7115

 Cleaning city names...
  After cleaning empty strings: 7115 rows

  Validating Late_delivery_risk column...
  Unique values: [1 0]
  Valid risk values: {1: 4189, 0: 2926}

 Handling Shipping Mode...
  Missing Shipping Mode: 0

 Handling duplicate routes...
  Duplicate routes removed: 0
  Remaining unique routes: 7115

 Data Cleaning Summary:
  Original rows: 7115
  Cleaned rows: 7115
  Rows removed: 0 (0.00%)
  Data retained: 100.00%

 Missing Values AFTER Cleaning:
Order Zipcode          6200
Product Description    7115
dtype: int64

Late Delivery Risk Distribution:
Late_delivery_risk
1    4189
0    2926
Name: count, dtype: int64
Late Risk Ratio: 58.88%


In [5]:
# ==========================================================================
# STEP 3: BUILD GRAPH STRUCTURE
# ============================================================================
print("\n" + "="*70)
print("[STEP 3] Building Graph from Order-Customer City Connections...")
print("="*70)

# Identify unique cities
all_cities = list(set(df['Order City'].tolist() + df['Customer City'].tolist()))
num_nodes = len(all_cities)
city_to_idx = {city: i for i, city in enumerate(all_cities)}
idx_to_city = {i: city for city, i in city_to_idx.items()}

print(f"\nTotal Unique Cities (Nodes): {num_nodes}")
print(f"Sample Cities: {all_cities[:10]}")

# Build directed graph
G = nx.DiGraph()
edge_attributes = []

for _, row in df.iterrows():
    src = city_to_idx[row['Order City']]
    dst = city_to_idx[row['Customer City']]
    late_risk = row['Late_delivery_risk']
    
    # Add edge with attributes
    if G.has_edge(src, dst):
        # If edge exists, aggregate (take max risk)
        G[src][dst]['late_risk'] = max(G[src][dst]['late_risk'], late_risk)
        G[src][dst]['count'] += 1
    else:
        G.add_edge(src, dst, late_risk=late_risk, count=1)

print(f"\nGraph Statistics:")
print(f"  Nodes: {G.number_of_nodes()}")
print(f"  Edges: {G.number_of_edges()}")
print(f"  Density: {nx.density(G):.4f}")
print(f"  Is Weakly Connected: {nx.is_weakly_connected(G)}")

# Identify isolated nodes
isolated_nodes = list(nx.isolates(G))
if len(isolated_nodes) > 0:
    print(f"  Isolated nodes: {len(isolated_nodes)}")
    print(f"  Removing isolated nodes...")
    G.remove_nodes_from(isolated_nodes)
    print(f"  Nodes after removal: {G.number_of_nodes()}")
    num_nodes = G.number_of_nodes()
    
    # Update mappings
    active_cities = [idx_to_city[i] for i in G.nodes()]
    city_to_idx = {city: i for i, city in enumerate(active_cities)}
    idx_to_city = {i: city for city, i in city_to_idx.items()}
    
    # Relabel graph
    mapping = {old: new for new, old in enumerate(G.nodes())}
    G = nx.relabel_nodes(G, mapping)

# Calculate network metrics
degree_centrality = nx.degree_centrality(G)
in_degree = dict(G.in_degree())
out_degree = dict(G.out_degree())

print(f"\nTop 5 Cities by Degree Centrality:")
top_cities = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
for idx, centrality in top_cities:
    print(f"  {idx_to_city[idx]}: {centrality:.4f} (In: {in_degree[idx]}, Out: {out_degree[idx]})")


[STEP 3] Building Graph from Order-Customer City Connections...

Total Unique Cities (Nodes): 2726
Sample Cities: ['New Orleans', 'Kediri', 'Itapecerica Da Serra', 'Flushing', 'Dordrecht', 'Broken Hill', 'Lanxi', 'Kisangani', 'Teziutlán', 'Mudanjiang']

Graph Statistics:
  Nodes: 2726
  Edges: 7115
  Density: 0.0010
  Is Weakly Connected: False

Top 5 Cities by Degree Centrality:
  Caguas: 0.5306 (In: 1446, Out: 0)
  Los Angeles: 0.0752 (In: 148, Out: 57)
  Chicago: 0.0723 (In: 168, Out: 29)
  Brooklyn: 0.0635 (In: 173, Out: 0)
  New York: 0.0385 (In: 105, Out: 0)


In [6]:
# =========================================================================
# STEP 4: CREATE NODE FEATURES
# ============================================================================
print("\n" + "="*70)
print("[STEP 4] Creating Node Features...")
print("="*70)

# Initialize feature matrix
feature_dim = 16
features = np.zeros((num_nodes, feature_dim))

# Feature engineering for each node
for node in range(num_nodes):
    # Feature 0-1: In/Out degree (normalized)
    max_in = max(in_degree.values()) if in_degree.values() else 1
    max_out = max(out_degree.values()) if out_degree.values() else 1
    features[node, 0] = in_degree.get(node, 0) / max_in
    features[node, 1] = out_degree.get(node, 0) / max_out
    
    # Feature 2: Degree centrality
    features[node, 2] = degree_centrality.get(node, 0)
    
    # Feature 3-4: Risk statistics (incoming/outgoing edges)
    incoming_risks = [G[u][node]['late_risk'] for u in G.predecessors(node)]
    outgoing_risks = [G[node][v]['late_risk'] for v in G.successors(node)]
    
    features[node, 3] = np.mean(incoming_risks) if incoming_risks else 0
    features[node, 4] = np.mean(outgoing_risks) if outgoing_risks else 0
    
    # Feature 5-6: Edge count statistics
    features[node, 5] = len(incoming_risks) / num_nodes  # Normalized
    features[node, 6] = len(outgoing_risks) / num_nodes  # Normalized
    
    # Features 7-15: Random embeddings (can be replaced with geographic/market features)
    features[node, 7:] = np.abs(np.random.randn(feature_dim - 7)) * 0.1

# Check for NaN or Inf in features
print(f"\n[4.1] Feature Quality Check:")
nan_count = np.isnan(features).sum()
inf_count = np.isinf(features).sum()
print(f"  NaN values in features: {nan_count}")
print(f"  Inf values in features: {inf_count}")

if nan_count > 0 or inf_count > 0:
    print(f"   Replacing NaN/Inf with zeros...")
    features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)

features = torch.FloatTensor(features)
print(f"\nNode Feature Matrix Shape: {features.shape}")
print(f"Feature Statistics:")
print(f"  Mean: {features.mean().item():.4f}")
print(f"  Std: {features.std().item():.4f}")
print(f"  Min: {features.min().item():.4f}")
print(f"  Max: {features.max().item():.4f}")


[STEP 4] Creating Node Features...

[4.1] Feature Quality Check:
  NaN values in features: 0
  Inf values in features: 0

Node Feature Matrix Shape: torch.Size([2726, 16])
Feature Statistics:
  Mean: 0.0868
  Std: 0.1737
  Min: 0.0000
  Max: 1.0000


In [7]:
# ==========================================================================
# STEP 5: CREATE NORMALIZED ADJACENCY MATRIX
# ============================================================================
print("\n" + "="*70)
print("[STEP 5] Creating Normalized Adjacency Matrix...")
print("="*70)

adj = nx.to_numpy_array(G, nodelist=range(num_nodes))
print(f"Adjacency Matrix Shape: {adj.shape}")
print(f"Number of Connections: {np.sum(adj > 0)}")

# Add self-loops for GNN stability
adj = adj + np.eye(num_nodes)
print(f"After adding self-loops: {np.sum(adj > 0)} connections")

# Normalize by degree (D^-1 * A)
degrees = np.sum(adj, axis=1)
degrees[degrees == 0] = 1  # Avoid division by zero

# Additional safety check
if np.any(degrees == 0):
    print("   Warning: Zero degrees found even after self-loops!")

adj_norm = adj / degrees[:, None]

# Check for NaN or Inf in adjacency
print(f"\n[5.1] Adjacency Matrix Quality Check:")
adj_nan = np.isnan(adj_norm).sum()
adj_inf = np.isinf(adj_norm).sum()
print(f"  NaN values: {adj_nan}")
print(f"  Inf values: {adj_inf}")

if adj_nan > 0 or adj_inf > 0:
    print(f"   Replacing NaN/Inf with zeros...")
    adj_norm = np.nan_to_num(adj_norm, nan=0.0, posinf=0.0, neginf=0.0)

adj_norm = torch.FloatTensor(adj_norm)

print(f"\nNormalized Adjacency Matrix Shape: {adj_norm.shape}")
print(f"Normalization check (row sums ≈ 1):")
print(f"  Min row sum: {adj_norm.sum(dim=1).min().item():.4f}")
print(f"  Max row sum: {adj_norm.sum(dim=1).max().item():.4f}")
print(f"  Mean row sum: {adj_norm.sum(dim=1).mean().item():.4f}")


[STEP 5] Creating Normalized Adjacency Matrix...
Adjacency Matrix Shape: (2726, 2726)
Number of Connections: 7115
After adding self-loops: 9836 connections

[5.1] Adjacency Matrix Quality Check:
  NaN values: 0
  Inf values: 0

Normalized Adjacency Matrix Shape: torch.Size([2726, 2726])
Normalization check (row sums ≈ 1):
  Min row sum: 1.0000
  Max row sum: 1.0000
  Mean row sum: 1.0000


In [8]:
# ==========================================================================
# STEP 6: PREPARE TRAINING DATA (EDGES AND LABELS)
# ============================================================================
print("\n" + "="*70)
print("[STEP 6] Preparing Edge Training Data...")
print("="*70)

edges = [(u, v) for u, v in G.edges()]
labels = [G[u][v]['late_risk'] for u, v in edges]

print(f"Total Edges: {len(edges)}")
print(f"Positive Labels (Late Risk=1): {sum(labels)} ({sum(labels)/len(labels):.2%})")
print(f"Negative Labels (Late Risk=0): {len(labels)-sum(labels)} ({(len(labels)-sum(labels))/len(labels):.2%})")

# Check for class imbalance
risk_ratio = sum(labels) / len(labels)
if risk_ratio < 0.1 or risk_ratio > 0.9:
    print(f"    Warning: Severe class imbalance detected ({risk_ratio:.2%})!")
    print(f"  Consider using class weights in loss function.")

# Split data
if len(edges) < 10:
    raise ValueError(f"Too few edges ({len(edges)}) for train/test split!")

edge_train, edge_test, label_train, label_test = train_test_split(
    edges, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"\nTrain Set: {len(edge_train)} edges")
print(f"  Positive: {sum(label_train)} ({sum(label_train)/len(label_train):.2%})")
print(f"Test Set: {len(edge_test)} edges")
print(f"  Positive: {sum(label_test)} ({sum(label_test)/len(label_test):.2%})")

# Convert to tensors
label_train_tensor = torch.FloatTensor(label_train)
label_test_tensor = torch.FloatTensor(label_test)



[STEP 6] Preparing Edge Training Data...
Total Edges: 7115
Positive Labels (Late Risk=1): 4189 (58.88%)
Negative Labels (Late Risk=0): 2926 (41.12%)

Train Set: 5692 edges
  Positive: 3351 (58.87%)
Test Set: 1423 edges
  Positive: 838 (58.89%)


In [12]:
# ============================================================================
# STEP 7: DEFINE GNN MODEL - FIXED VERSION
# ============================================================================
print("\n" + "="*70)
print("[STEP 7] Defining FIXED Graph Neural Network Architecture...")
print("="*70)

class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        nn.init.xavier_uniform_(self.weight)
    
    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support)
        return F.relu(output)

class SupplyChainGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SupplyChainGNN, self).__init__()
        self.gcn1 = GCNLayer(input_dim, hidden_dim)
        self.gcn2 = GCNLayer(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)
        
        #  FIX: Add proper edge predictor MLP
        self.edge_predictor = nn.Sequential(
            nn.Linear(output_dim * 2, hidden_dim),  # Concatenate src + dst embeddings
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1)  # Output single logit
        )
    
    def forward(self, x, adj):
        x = self.gcn1(x, adj)
        x = self.dropout(x)
        x = self.gcn2(x, adj)
        return x
    
    def predict_edge(self, embeds, src, dst):
        """ FIXED: Use MLP for edge prediction instead of dot product"""
        src_embed = embeds[src]
        dst_embed = embeds[dst]
        
        # Concatenate source and destination embeddings
        edge_repr = torch.cat([src_embed, dst_embed], dim=0)
        
        # Pass through MLP and apply sigmoid
        logit = self.edge_predictor(edge_repr)
        return torch.sigmoid(logit.squeeze())
    
    def predict_edges_batch(self, embeds, edge_list):
        """ FIXED: Vectorized batch prediction with MLP"""
        src_indices = torch.LongTensor([src for src, dst in edge_list])
        dst_indices = torch.LongTensor([dst for src, dst in edge_list])
        
        src_embeds = embeds[src_indices]  # (num_edges, embed_dim)
        dst_embeds = embeds[dst_indices]  # (num_edges, embed_dim)
        
        # Concatenate along feature dimension
        edge_reprs = torch.cat([src_embeds, dst_embeds], dim=1)  # (num_edges, embed_dim*2)
        
        # Pass through MLP
        logits = self.edge_predictor(edge_reprs)  # (num_edges, 1)
        return torch.sigmoid(logits.squeeze())

model = SupplyChainGNN(input_dim=16, hidden_dim=32, output_dim=8)
print(f"\n FIXED Model Architecture:")
print(model)
print(f"\nTotal Parameters: {sum(p.numel() for p in model.parameters())}")


[STEP 7] Defining FIXED Graph Neural Network Architecture...

 FIXED Model Architecture:
SupplyChainGNN(
  (gcn1): GCNLayer()
  (gcn2): GCNLayer()
  (dropout): Dropout(p=0.3, inplace=False)
  (edge_predictor): Sequential(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): ReLU()
    (5): Linear(in_features=16, out_features=1, bias=True)
  )
)

Total Parameters: 1857


In [21]:
# ============================================================================
# STEP 8: TRAINING LOOP 
# ============================================================================
print("\n" + "="*70)
print("[STEP 8] Training the FIXED Model...")
print("="*70)

optimizer = Adam(model.parameters(), lr=0.005, weight_decay=1e-4)  # Lower LR for stability
loss_fn = nn.BCELoss()
num_epochs = 300

#  
pos_weight = (len(label_train) - sum(label_train)) / sum(label_train) if sum(label_train) > 0 else 1.0
loss_fn_weighted = nn.BCELoss(reduction='none')

train_losses = []
test_losses = []
train_accs = []
test_accs = []

print("\n FIXED Training Configuration:")
print(f"  Learning Rate: 0.005")
print(f"  Weight Decay: 1e-4")
print(f"  Epochs: {num_epochs}")
print(f"  Dropout: 0.3")
print(f"  Class Weight (positive): {pos_weight:.2f}")
print(f"  Using MLP Edge Predictor (no more saturation!)")

import time
start_time = time.time()
best_test_loss = float('inf')
patience = 50
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    embeds = model(features, adj_norm)
    
    if torch.isnan(embeds).any():
        print(f"    NaN detected in embeddings at epoch {epoch}!")
        break
    
    train_preds = model.predict_edges_batch(embeds, edge_train)
    
   
    losses = loss_fn_weighted(train_preds, label_train_tensor)
    weights = torch.where(label_train_tensor == 1, pos_weight, 1.0)
    loss = (losses * weights).mean()
    
    # L2 regularization on embeddings
    embedding_reg = 0.0001 * torch.norm(embeds, p=2)
    loss = loss + embedding_reg
    
    if torch.isnan(loss):
        print(f"    NaN loss at epoch {epoch}!")
        break
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    
    train_losses.append(loss.item())
    
    train_preds_binary = (train_preds.detach() > 0.5).float()
    train_acc = (train_preds_binary == label_train_tensor).float().mean().item()
    train_accs.append(train_acc)
    
    # Evaluate on test set
    if epoch % 20 == 0:
        model.eval()
        with torch.no_grad():
            test_embeds = model(features, adj_norm)
            test_preds = model.predict_edges_batch(test_embeds, edge_test)
            test_loss = loss_fn(test_preds, label_test_tensor).item()
            test_losses.append(test_loss)
            
            test_preds_binary = (test_preds > 0.5).float()
            test_acc = (test_preds_binary == label_test_tensor).float().mean().item()
            test_accs.append(test_acc)
            
            print(f"Epoch {epoch:3d} | Train Loss: {loss.item():.4f} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")
            
            # Early stopping
            if test_loss < best_test_loss:
                best_test_loss = test_loss
                patience_counter = 0
            else:
                patience_counter += 1
            
            if patience_counter >= patience:
                print(f"\n   Early stopping at epoch {epoch} (no improvement for {patience} checks)")
                break

training_time = time.time() - start_time
print(f"\n Training Complete! Time: {training_time:.2f}s")



[STEP 8] Training the FIXED Model...

 FIXED Training Configuration:
  Learning Rate: 0.005
  Weight Decay: 1e-4
  Epochs: 300
  Dropout: 0.3
  Class Weight (positive): 0.70
  Using MLP Edge Predictor (no more saturation!)
Epoch   0 | Train Loss: 0.4775 | Train Acc: 0.6557 | Test Loss: 0.5586 | Test Acc: 0.6732
Epoch  20 | Train Loss: 0.4724 | Train Acc: 0.6537 | Test Loss: 0.5939 | Test Acc: 0.6304
Epoch  40 | Train Loss: 0.4752 | Train Acc: 0.6667 | Test Loss: 0.5689 | Test Acc: 0.6578
Epoch  60 | Train Loss: 0.4757 | Train Acc: 0.6636 | Test Loss: 0.5688 | Test Acc: 0.6535
Epoch  80 | Train Loss: 0.4686 | Train Acc: 0.6636 | Test Loss: 0.5743 | Test Acc: 0.6486
Epoch 100 | Train Loss: 0.4693 | Train Acc: 0.6528 | Test Loss: 0.5721 | Test Acc: 0.6465
Epoch 120 | Train Loss: 0.4661 | Train Acc: 0.6609 | Test Loss: 0.5775 | Test Acc: 0.6409
Epoch 140 | Train Loss: 0.4784 | Train Acc: 0.6600 | Test Loss: 0.5644 | Test Acc: 0.6550
Epoch 160 | Train Loss: 0.4737 | Train Acc: 0.6528 | Tes

In [ ]:
# ============================================================================
# STEP 9: EVALUATION 
# ============================================================================
print("\n" + "="*70)
print("[STEP 9] Model Evaluation...")
print("="*70)

model.eval()
with torch.no_grad():
    final_embeds = model(features, adj_norm)
    test_preds_prob = model.predict_edges_batch(final_embeds, edge_test).numpy()
    test_preds_binary = (test_preds_prob > 0.5).astype(int)

acc = accuracy_score(label_test, test_preds_binary)
auc = roc_auc_score(label_test, test_preds_prob)

print(f"\n Test Set Performance:")
print(f"  Accuracy: {acc:.4f}")
print(f"  AUC-ROC: {auc:.4f}")

print(f"\n Prediction Distribution Analysis:")
print(f"  Min Prediction: {test_preds_prob.min():.4f}")
print(f"  Max Prediction: {test_preds_prob.max():.4f}")
print(f"  Mean Prediction: {test_preds_prob.mean():.4f}")
print(f"  Std Prediction: {test_preds_prob.std():.4f}")
print(f"  Median Prediction: {np.median(test_preds_prob):.4f}")

saturated_high = (test_preds_prob > 0.95).sum()
saturated_low = (test_preds_prob < 0.05).sum()
print(f"\n Saturation Check (should be MUCH better now):")
print(f"  Predictions > 0.95: {saturated_high} ({saturated_high/len(test_preds_prob)*100:.1f}%)")
print(f"  Predictions < 0.05: {saturated_low} ({saturated_low/len(test_preds_prob)*100:.1f}%)")
print(f"  Well-calibrated (0.05-0.95): {len(test_preds_prob)-saturated_high-saturated_low} ({(len(test_preds_prob)-saturated_high-saturated_low)/len(test_preds_prob)*100:.1f}%)")

print(f"\nClassification Report:")
print(classification_report(label_test, test_preds_binary, 
                          target_names=['No Risk', 'Late Risk'], zero_division=0))

print(f"\nConfusion Matrix:")
cm = confusion_matrix(label_test, test_preds_binary)
print(cm)
print(f"\nConfusion Matrix Breakdown:")
print(f"  True Negatives (Correct No Risk): {cm[0,0]}")
print(f"  False Positives (Predicted Risk, Actually No Risk): {cm[0,1]}")
print(f"  False Negatives (Predicted No Risk, Actually Risk): {cm[1,0]}")
print(f"  True Positives (Correct Late Risk): {cm[1,1]}")


[STEP 9] Model Evaluation...

 Test Set Performance:
  Accuracy: 0.6613
  AUC-ROC: 0.7552

 Prediction Distribution Analysis:
  Min Prediction: 0.0044
  Max Prediction: 0.9592
  Mean Prediction: 0.5021
  Std Prediction: 0.2080
  Median Prediction: 0.4957

 Saturation Check (should be MUCH better now):
  Predictions > 0.95: 14 (1.0%)
  Predictions < 0.05: 68 (4.8%)
  Well-calibrated (0.05-0.95): 1341 (94.2%)

Classification Report:
              precision    recall  f1-score   support

     No Risk       0.57      0.72      0.64       585
   Late Risk       0.76      0.62      0.68       838

    accuracy                           0.66      1423
   macro avg       0.66      0.67      0.66      1423
weighted avg       0.68      0.66      0.66      1423


Confusion Matrix:
[[420 165]
 [317 521]]

Confusion Matrix Breakdown:
  True Negatives (Correct No Risk): 420
  False Positives (Predicted Risk, Actually No Risk): 165
  False Negatives (Predicted No Risk, Actually Risk): 317
  True Pos

In [20]:
#= ===========================================================================
# STEP 10: RISK ANALYSIS & INSIGHTS
# ============================================================================
print("\n" + "="*70)
print("[STEP 10] Supply Chain Risk Insights...")
print("="*70)

node_risk_scores = torch.norm(final_embeds, dim=1).numpy()
high_risk_nodes = np.argsort(node_risk_scores)[-10:]

print(f"\nTop 10 High-Risk Cities (by embedding magnitude):")
for rank, node_idx in enumerate(high_risk_nodes[::-1], 1):
    city_name = idx_to_city[node_idx]
    risk_score = node_risk_scores[node_idx]
    in_deg = in_degree.get(node_idx, 0)
    out_deg = out_degree.get(node_idx, 0)
    print(f"  {rank}. {city_name}: Risk={risk_score:.4f}, In={in_deg}, Out={out_deg}")

edge_risks = [(edge, prob) for edge, prob in zip(edge_test, test_preds_prob)]
edge_risks_sorted = sorted(edge_risks, key=lambda x: x[1], reverse=True)

print(f"\nTop 10 Highest Risk Shipping Routes:")
for rank, ((src, dst), prob) in enumerate(edge_risks_sorted[:10], 1):
    src_city = idx_to_city[src]
    dst_city = idx_to_city[dst]
    actual = label_test[edge_test.index((src, dst))]
    match =  "|"if (prob > 0.5 and actual == 1) or (prob <= 0.5 and actual == 0) else "✗"
    print(f"  {rank}. {src_city} → {dst_city}: Predicted={prob:.4f}, Actual={actual} {match}")


[STEP 10] Supply Chain Risk Insights...

Top 10 High-Risk Cities (by embedding magnitude):
  1. Lanxi: Risk=1.0439, In=0, Out=1
  2. Acarigua: Risk=1.0082, In=0, Out=1
  3. Yaroslavl: Risk=1.0029, In=0, Out=1
  4. Halmstad: Risk=1.0028, In=0, Out=1
  5. Beirut: Risk=0.9568, In=0, Out=1
  6. Springdale: Risk=0.9388, In=0, Out=1
  7. Puerto Montt: Risk=0.9114, In=0, Out=1
  8. Woodland: Risk=0.8940, In=0, Out=1
  9. Pitesti: Risk=0.8800, In=0, Out=1
  10. Kitwe: Risk=0.8764, In=0, Out=1

Top 10 Highest Risk Shipping Routes:
  1. Belfort → Caguas: Predicted=0.9592, Actual=1 |
  2. Abha → Caguas: Predicted=0.9576, Actual=1 |
  3. Potenza → Caguas: Predicted=0.9569, Actual=1 |
  4. Carquefou → Caguas: Predicted=0.9549, Actual=1 |
  5. Arnhem → Caguas: Predicted=0.9548, Actual=1 |
  6. Lagos De Moreno → Caguas: Predicted=0.9541, Actual=1 |
  7. Lausanne → Caguas: Predicted=0.9532, Actual=1 |
  8. Ajaccio → Caguas: Predicted=0.9531, Actual=1 |
  9. Elazig → Caguas: Predicted=0.9527, Actual=1